In [ ]:
import torch

import arviz as az

from autoemulate.experimental.simulations.projectile import Projectile
from autoemulate.experimental.compare import AutoEmulate
from autoemulate.experimental.calibration.bayes import BayesianCalibration
from autoemulate.experimental.emulators.gaussian_process.exact import (
    GaussianProcessExact,
)

# Bayesian calibration

Bayesian calibration estimates the probability distribution over input parameters
given observed data, providing uncertainty estimates.

Performing Bayesian calibration requires:
- a fit emulator that predicts uncertainty (e.g., Gaussian Process) and
- observations associated with the simulator output.



## 1. Simulate data and train an emulator

In [ ]:
simulator = Projectile()
x = simulator.sample_inputs(100)
y = simulator.forward_batch(x)

In [ ]:
ae = AutoEmulate(x, y, models=[GaussianProcessExact])

In [ ]:
ae.summarise()

In [ ]:
model = ae.best_result().model

## 2. Calibrate

We start with a set of noisy, experimental observations. 

Below we pick some initial parameter values and simulate the output. We then add noise to generate 10 pseudo observations.

In [ ]:
true_velocity = 50  # m/s
true_drag_coefficient = 0.1

# simulator expects inputs of shape [1, number of inputs]
params = torch.tensor([true_drag_coefficient, true_velocity]).view(1, -1)
true_distance = simulator.forward(params)

n_obs = 10
noise = torch.normal(mean=0, std=1, size=(n_obs,))
observed_distances = true_distance[0] + noise

observed_distances.shape

We set up the `BayesianCalibration` object with out trained emulator, the simulator parameter ranges and the "observed data". 

In [ ]:
observations = {"distance": observed_distances}
observation_noise = 1.0

bc = BayesianCalibration(
    model, 
    simulator.parameters_range, 
    observations, 
    observation_noise
)

Run MCMC using the NUTS sampler.

In [ ]:
mcmc = bc.run_mcmc(
    warmup_steps=250, 
    num_samples=1000,
    sampler='nuts',
    )

The `run_mcmc` method returns the Pyro MCMC object which has a number of useful methods associated with it. One can access all the posterior samples using `mcmc.get_samples()` or access just the summary statistics using `mcmc.summary()`.

In [ ]:
mcmc.summary()

## 3. Plotting with Arviz

The `BayesianCalibrator` object can be used to make the `mcmc` object compatible witg the Arviz plotting library. It is then very easy to produce all the typical plots of the results as well as MCMC diagnostics.

In [ ]:
az_data = bc.to_arviz(mcmc, posterior_predictive=True)

In [ ]:
az.plot_pair(az_data, kind='kde')

In [ ]:
az.plot_ppc(az_data, kind='scatter')

In [ ]:
az.plot_trace(az_data)

In [ ]:
az.plot_autocorr(az_data)